### Importowanie bibliotek


In [10]:
import requests
from bs4 import BeautifulSoup

### Wyslanie żadania dostepu do strony internetowej z opiniami o produkcie 

In [6]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code

200

### Jeżeli operacja zakończy sie powodzeniem

In [13]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

### Dla każdej opinii wyodrębnienie z kodu HTML poszzczególnych skladowych i przypisanie ich do elementów złożonej struktury danych

In [30]:
for opinion in opinions:
    try:
        single_opinion = {
            "author": opinion.select_one("span.user-post__author-name").text.strip(),
            "recomendation":opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
            "stars":opinion.select_one("span.user-post__score-count").text.strip(),
            "content":opinion.select_one("div.user-post__text").text.strip(),
            "pros":(p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")),
            "cons":(c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")),
            "useful":opinion.select_one("button.vote-yes > span").text.strip(),
            "useless":opinion.select_one("button.vote-no > span").text.strip(),
            "post_date":opinion.select_one("span.user-post__published > time:nth-child(1)[datetime]").text.strip(),
            "purchase_date":opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]").text.strip(),
        }
        print(single_opinion)
    except (AttributeError, TypeError):
        pass

{'author': 'b...a', 'recomendation': 'Polecam', 'stars': '4/5', 'content': 'Wykonana dobrze, trochę za duża, z ledwością mieści mi się na biurku, ale może być. Drukuje ciszej w porównaniu z poprzednią drukarką HP 209a. Napełniłam tusze do pełna i po ok.10 kartkach ubyło mi 4 mm czarnego tuszu w pojemniku. Mam nadzieję, że będzie wydajna. Na razie nie mogę tego stwierdzić. Fajnie, że można zobaczyć, ile ubyło tuszu. Dolewanie jest banalnie proste, tusz nie wylewa się. Martwi mnie tylko, że przenosić trzeba ją w pozycji poziomej, bo inaczej tusz może się wylewać. Ogólnie jestem zadowolona z zakupu.', 'pros': <generator object <genexpr> at 0x00000191917D1220>, 'cons': None, 'useful': '2', 'useless': '3', 'post_date': '5 lat temu,', 'purchase_date': 'po 6 dniach'}
{'author': 'U...o', 'recomendation': 'Polecam', 'stars': '4,5/5', 'content': 'Po 1 dniu uzytkowania mozna stwierdzic 1-polaczenie z kompem poprzez wi fi-bez problemu\n2-tankowanie zbiorniczkow-bez problemu\n3-montaz ,,glowic,,-be

In [17]:
print(type(single_opinion["author"]))

<class 'str'>
